In [1]:
from ultralytics import YOLO
import os
import shutil
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load a model
model = YOLO('yolov8x-pose.pt')  # load a pretrained model

# Prompt user for the video name
video_name = input("Введите название видео (без расширения .mp4): ")

# Construct input and output file paths
input_path = f'./data/videos/input/{video_name}.mp4'
output_path = f'./data/videos/output/{video_name}_output.mp4'

# Ensure the output directory exists
os.makedirs('./data/videos/output/', exist_ok=True)

# Predict with model on the video, specifying the output directory
output_dir = './data/videos/output/'
# model.predict(source=input_path, save=True, save_txt=True, save_conf=True, project=output_dir, name=video_name+'_output', show=True, conf=0.05)
model.predict(source=input_path, save=True, save_txt=True, save_conf=True, project=output_dir, name=video_name+'_output', show=True, conf=0.2)

# Find the most recent output directory
output_dirs = sorted([d for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d))], key=lambda x: os.path.getctime(os.path.join(output_dir, x)), reverse=True)

if not output_dirs:
    print("Не найдена директория с результатами. Проверьте вывод YOLO.")
else:
    latest_output_dir = os.path.join(output_dir, output_dirs[0])

    # Find the output video file
    output_files = [f for f in os.listdir(latest_output_dir) if f.endswith('.mp4')]
    if not output_files:
        print("Не найдено видео в результатах. Проверьте вывод YOLO.")
    else:
        latest_output_file = os.path.join(latest_output_dir, output_files[0])

        # Move the output file to the desired location
        final_output_path = os.path.join(output_dir, video_name + '_output.mp4')
        shutil.move(latest_output_file, final_output_path)
        print(f"Обработанное видео сохранено по пути {final_output_path}")




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/56) D:\Users\Legion\BIA Technologies\YOLO_8_test\data\videos\input\12-07-2024.mp4: 384x640 1 person, 129.3ms
video 1/1 (frame 2/56) D:\Users\Legion\BIA Technologies\YOLO_8_test\data\videos\input\12-07-2024.mp4: 384x640 1 person, 42.7ms
video 1/1 (frame 3/56) D:\Users\Legion\BIA Technologies\YOLO_8_test\data\videos\input\12-07-2024.mp4: 384x640 2 persons, 43.0ms
video 1/1 (frame 4/56) D:\Users\Legion\BIA Technologies\YOLO_8_test\data\vi

In [3]:
import cv2

#get the dimension of the video
cap = cv2.VideoCapture('video_path')
if cap.isOpened():
    w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    size = (w, h)
    print(size)

In [4]:
out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), cap.get(cv2.CAP_PROP_FPS), size)

NameError: name 'size' is not defined

In [1]:
!pip freeze


absl-py==2.1.0
anyio==4.4.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
async-lru==2.0.4
attrs==23.2.0
Babel==2.15.0
beautifulsoup4==4.12.3
bleach==6.1.0
certifi==2024.6.2
cffi==1.16.0
charset-normalizer==3.3.2
colorama==0.4.6
comm==0.2.2
contourpy==1.2.1
cvlib==0.2.7
cycler==0.12.1
debugpy==1.8.1
decorator==5.1.1
defusedxml==0.7.1
executing==2.0.1
fastjsonschema==2.20.0
filelock==3.15.3
flatbuffers==24.3.25
fonttools==4.53.0
fqdn==1.5.1
fsspec==2024.6.0
gitdb==4.0.11
GitPython==3.1.43
h11==0.14.0
httpcore==1.0.5
httpx==0.27.0
idna==3.7
imageio==2.34.2
imutils==0.5.4
intel-openmp==2021.4.0
ipykernel==6.29.4
ipython==8.25.0
ipywidgets==8.1.3
isoduration==20.11.0
jax==0.4.30
jaxlib==0.4.30
jedi==0.19.1
Jinja2==3.1.4
json5==0.9.25
jsonpointer==3.0.0
jsonschema==4.22.0
jsonschema-specifications==2023.12.1
jupyter==1.0.0
jupyter-console==6.6.3
jupyter-events==0.10.0
jupyter-lsp==2.2.5
jupyter_client==8.6.2
jupyter_core==5.7.2
jupyter_server==2.14.1
jupyter

In [7]:
import cv2
import numpy as np
from ultralytics import YOLO
import os

# Load YOLOv8 model
model = YOLO('yolov8x-pose.pt')

def process_video(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Prepare output video writer
    out = cv2.VideoWriter(os.path.join(output_dir, 'output_skeleton.mp4'),
                          cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
    
    frames_data = []
    frame_id = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Run YOLOv8 inference on the frame
        results = model(frame)
        
        # Process results and draw skeletons
        frame_data = process_results(results, frame, frame_id)
        frames_data.append(frame_data)
        
        # Write frame with skeletons to video
        out.write(frame)
        
        frame_id += 1
    
    cap.release()
    out.release()
    
    # Write skeleton data to file
    write_skeleton_file(os.path.join(output_dir, 'S001C001P001R001A060.skeleton'), frames_data)

def process_results(results, frame, frame_id):
    frame_data = {
        'frame_id': frame_id,
        'bodies': []
    }
    
    for person_id, person in enumerate(results[0].keypoints.xy):
        body_data = {
            'body_id': person_id,
            'joints': []
        }
        
        for joint_id, joint in enumerate(person):
            x, y = joint
            # Add more properties as needed (Z, color, depth, etc.)
            joint_data = {
                'x': float(x),
                'y': float(y),
                'z': 0.0,  # Placeholder, adjust as needed
                'color_x': 0,  # Placeholder
                'color_y': 0,  # Placeholder
                'depth': 0,  # Placeholder
                'ir': 0,  # Placeholder
                'normal_x': 0,  # Placeholder
                'normal_y': 0,  # Placeholder
                'normal_z': 0,  # Placeholder
                'extra': 0  # Placeholder
            }
            body_data['joints'].append(joint_data)
            
            # Draw skeleton on frame
            cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)
        
        frame_data['bodies'].append(body_data)
        
    return frame_data

def write_skeleton_file(filename, frames_data):
    with open(filename, 'w') as f:
        f.write(f"{len(frames_data)}\n")  # Number of frames
        for frame in frames_data:
            f.write(f"{len(frame['bodies'])}\n")  # Number of bodies in frame
            for body in frame['bodies']:
                # Write body data
                f.write(f"{frame['frame_id']} {body['body_id']} 1 1 1 1 0 0 0 2\n")
                # Write joint data
                for joint in body['joints']:
                    f.write(f"{joint['x']} {joint['y']} {joint['z']} {joint['color_x']} {joint['color_y']} "
                            f"{joint['depth']} {joint['ir']} {joint['normal_x']} {joint['normal_y']} "
                            f"{joint['normal_z']} {joint['extra']} 2\n")

# Main execution
video_path = "D:/Users/Legion/BIA Technologies/YOLO_8_test/data/videos/output/25-06-2024_output3/25-06-2024.avi"
output_dir = 'D:/Users/Legion/BIA Technologies/dataset_SGN/output_labels'

process_video(video_path, output_dir)


0: 384x640 3 persons, 38.0ms
Speed: 1.0ms preprocess, 38.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 26.9ms
Speed: 1.1ms preprocess, 26.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 30.0ms
Speed: 2.2ms preprocess, 30.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 32.9ms
Speed: 1.8ms preprocess, 32.9ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 33.3ms
Speed: 1.0ms preprocess, 33.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 30.4ms
Speed: 1.0ms preprocess, 30.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 30.0ms
Speed: 2.0ms preprocess, 30.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 29.9ms
Speed: 2.0ms preprocess, 29.9ms inference, 2.0ms postprocess per image at shape 